# 准备IMDB影评数据

In [7]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers


batch_size = 32
train_ds = keras.utils.text_dataset_from_directory('aclImdb/train', batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory('aclImdb/val', batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test', batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [2]:
for inputs, targets in train_ds:
    print("inputs.shape: ", inputs.shape)
    print("inputs.dtype: ", inputs.dtype)
    print("targets.shape: ", targets.shape)
    print("targets.dtype: ", targets.dtype)
    print("inputs[0]: " , inputs[0])
    print("targets[0]: ", targets[0])
    break

inputs.shape:  (32,)
inputs.dtype:  <dtype: 'string'>
targets.shape:  (32,)
targets.dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor(b'I am sad that a period of history that is so fascinating and so rich in material for film can be made into a ho-hum event . Wm C Quantrill was barely shown in the film , probably the most intriquing figure of the period. Frank James was never mentioned, Cole Younger , ditto , and Bloody Bill Anderson , who would weep for his murdered sister every time he went into battle was completely absent in the script. Instead we were forced to watch fictitious characters that never developed into anyone we cared about. how sad. The costumes were wonderful however, as was the location shooting in Missouri. I hope Ang Lee will make another film from the period and try again, or some other film maker will look into the tremendous wealth of material to write a screen play on .', shape=(), dtype=string)
targets[0]:  tf.Tensor(0, shape=(), dtype=int32)


In [3]:
text_vectorization = TextVectorization(max_tokens=20000, output_mode='multi_hot')
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape: ", inputs.shape)
    print("inputs.dtype: ", inputs.dtype)
    print("targets.shape: ", targets.shape)
    print("targets.dtype: ", targets.dtype)
    print("inputs[0]: " , inputs[0])
    print("targets[0]: ", targets[0])
    break

inputs.shape:  (32, 20000)
inputs.dtype:  <dtype: 'float32'>
targets.shape:  (32,)
targets.dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]:  tf.Tensor(1, shape=(), dtype=int32)


# 词袋方法
1. 单个单词的二进制编码

In [4]:
def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation='relu')(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model


model = get_model()
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint('binary_1gram.keras', save_best_only=True)]
model.fit(binary_1gram_train_ds.cache(), validation_data=binary_1gram_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model('binary_1gram.keras')
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 21s 32ms/step - loss: 0.3916 - accuracy: 0.8388 - val_loss: 0.2947 - val_accuracy: 0.8816
Epoch 2/10
625/625 [=======================

2. 二元语法的二进制编码

In [5]:
text_vectorization = TextVectorization(ngrams=2, max_tokens=20000, output_mode='multi_hot')
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
callbacks = [keras.callbacks.ModelCheckpoint('binary_2gram.keras', save_best_only=True)]
model.fit(binary_2gram_train_ds.cache(), validation_data=binary_2gram_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model('binary_2gram.keras')
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 23s 36ms/step - loss: 0.4760 - accuracy: 0.7912 - val_loss: 0.3344 - val_accuracy: 0.8730
Epoch 2/10
625/625 [==============================] - 25s 40ms/step - loss: 0.2910 - accuracy: 0.8932 - val_loss: 0.3217 - val_accuracy: 0.8906
Epoch 3/10
625/625 [==============================] - 25s 41ms/step - loss: 0.2435 - accuracy: 0.9171 - val_loss: 0.3226 - val_accuracy: 0.8914
Epoch 4/10
625/625 [==============================] - 27s 43ms/step - loss: 0.2247 - accuracy: 0.9273 - val_loss: 0.3474 - val_accuracy: 0.8916
Epoch 5/10
625/625 [==============================] - 25s 40ms/step - loss: 0.2091 - accuracy: 0.9347 - val_loss: 0.3604 - val_accuracy: 0.8916
Epoch 6/10
625/625 [==============================] - 25s 39ms/step - loss: 0.2153 - accuracy: 0.9359 - val_loss: 0.3662 - val_accuracy: 0.8906
Epoch 7/10
625/625 [==============================] - 25s 41ms/step - loss: 0.2030 - accuracy: 0.9373 - val_loss: 0.3770 - val_accuracy:

3. 二元语法的TF-IDF编码

In [ ]:
text_vectorization = TextVectorization(ngrams=2, max_tokens=20000, output_mode='tf-idf')
text_vectorization.adapt(text_only_train_ds)
tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
callbacks = [keras.callbacks.ModelCheckpoint('tfidf_2gram.keras', save_best_only=True)]
model.fit(tfidf_2gram_train_ds.cache(), validation_data=tfidf_2gram_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model('tfidf_2gram.keras')
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 25s 39ms/step - loss: 0.7710 - accuracy: 0.8906 - val_loss: 0.3621 - val_accuracy: 0.8830
Epoch 2/10
625/625 [==============================] - 26s 41ms/step - loss: 0.2674 - accuracy: 0.9042 - val_loss: 0.3954 - val_accuracy: 0.8834
Epoch 3/10
625/625 [==============================] - 26s 41ms/step - loss: 0.2480 - accuracy: 0.9068 - val_loss: 0.4037 - val_accuracy: 0.8852
Epoch 4/10
625/625 [==============================] - 2092s 3s/step - loss: 0.2366 - accuracy: 0.9072 - val_loss: 0.3714 - val_accuracy: 0.8838
Epoch 5/10
625/625 [==============================] - 27s 43ms/step - loss: 0.2286 - accuracy: 0.9100 - val_loss: 0.3796 - val_accuracy: 0.8786
Epoch 6/10
521/625 [========================>.....] - ETA: 3s - loss: 0.2237 - accuracy: 0.9082

In [ ]:
inputs = keras.Input(shape=(1,), dtype='string')
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

raw_text_data = tf.convert_to_tensor([["That was an excellent movie, I love it."]])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.3f} percent positive")

# 序列模型方法
1. 第一个实例

In [4]:
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=max_length)
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

inputs = keras.Input(shape=(None,), dtype='int64')
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint('one_hot_bidir_lstm.keras', save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

2. 利用Embedding层学习词嵌入

In [ ]:
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint('embeddings_bidir_gru.keras', save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model('embeddings_bidir_gru.keras')
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

3. 理解填充和掩码

In [ ]:
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model('embeddings_bidir_gru_with_masking.keras')
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_4 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

4. 使用预训练的词嵌入

In [27]:
path_to_glove_file = 'glove.6B.100d.txt'
embeddings_index = {}

with open(path_to_glove_file, encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [28]:
embedding_dim = 100
vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))
embedding_matrix = np.zeros((max_tokens, embedding_dim))

for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = layers.Embedding(max_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False, mask_zero=True)
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint('glove_embeddings_sequence_model.keras', save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model('glove_embeddings_sequence_model.keras')
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________

 [返回](text_data_preprocessing.md)